# Calling Predict API on Model Hosted Outside of Fiddler
Here, we will walkthrough how to use a model that is hosted external to Fiddler.

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.

In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Dataset Upload
To upload a model, you first need to upload a sample of the data of the model’s 
inputs, targets, and additional metadata that might be useful for model analysis. 
This data sample helps us (among other things) to infer the model schema and the 
data types and values range of each feature.

In [ ]:
import pandas as pd

# If dataset is already in Fiddler, just use it.
if 'wine_quality' in client.list_datasets():
    df_schema = client.get_dataset_info('wine_quality')
    df = client.get_dataset('wine_quality')['train']
else:
    # Otherwise upload it.
    df = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/train.csv')
    df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)
    upload_result = client.upload_dataset(
        dataset={'train': df}, 
        dataset_id='wine_quality')

## Create Model Schema
As you must have noted, in the dataset upload step we did not ask for the model’s 
features and targets, or any model specific information. That’s because we 
allow for linking multiple models to a given dataset schema. Hence we require 
an Infer model schema step which helps us know the features relevant to the 
model and the model task. Here you can specify the input features, the target 
column, decision columns and metadata columns, and also the type of model.

In [ ]:
target = 'quality'

feature_columns = df_schema.get_column_names()
feature_columns.remove('row_id')
feature_columns.remove(target)

model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=df_schema,
    target=target, 
    features=feature_columns,
    display_name='external model',
    description='this is an external model called from fiddler via rest API'
)

# Save Model Schema
Next step, we need to save the model and any pre-processing step you had 
on the input features (for example Categorical encoder, Tokenization, ...).

In [ ]:
import pathlib
import shutil
import yaml

model_dir = pathlib.Path('external_model')
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)


## Write `package.py`
A wrapper is needed between Fiddler and the model. This wrapper can be used to 
translate the inputs and outputs to fit what the model expects and what Fiddler 
is able to consume. More information can be found [here](https://docs.fiddler.ai/api-reference/package-py/)

In [ ]:
%%writefile external_model/package.py

from pathlib import Path
import pandas as pd
import requests
import json
import logging

# make sure model generated in tutorial 02 is deployed to server
external_model_endpoint = 'http://host.docker.internal:5100/execute/onebox/tutorial/wine_quality_model'


class ExternalModelPackage:
    is_classifier = False

    def predict(self, input_df):
        logging.info(f'input df: {input_df}')
        # convert input datafrme to a format that external model accepts
        data_array = [y.iloc[0,:].to_dict() for x , y in input_df.groupby(level=0)]
        data = dict(data=data_array)
        json_input = json.dumps(data)

        # call external service
        json_result = self.invoke_external_model(json_input)
        
        # convert response back to dataframe
        return pd.DataFrame(json_result)
    
    # invoke the externa model using API
    def invoke_external_model(self, json_input):
        logging.info(f'input json: {json_input}')
        headers = {'Content-type': 'application/json'}
        result = requests.post(external_model_endpoint, 
                               headers=headers, 
                               data=json_input)
        logging.info(f'result: {result}')
        return result.json()['result']
    
def get_model():
    return ExternalModelPackage()


# Upload Model
Now that we have all the parts that we need, we can go ahead and upload the model to the Fiddler platform. You can use the [upload_model_package](https://docs.fiddler.ai/api-reference/python-package/#upload-model-package) to upload this entire directory in one shot. We need the following for uploading a model:
- The `path` to the directory
- The `project_id` to which the model belongs
- The `model_id`, which is the name you want to give the model. You can access it in Fiddler henceforth via this ID
- The `dataset` which the model is linked to (optional)  

In total, we will have a `model.yaml` and a `package.py` file within our model directory.

In [ ]:
project_id = 'tutorial'
model_id = 'external_model'
client.delete_model(project_id, model_id)
client.upload_model_package(model_dir, project_id, model_id)

# Test Model
Now, let's test out our model by interfacing with the client and 
calling [run model](https://docs.fiddler.ai/api-reference/python-package/#run-model).

In [ ]:
client.run_model(project_id, model_id, df[0:20])